# **MIP4_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7"""

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [6]:
print(response1.choices[0].message.content)

Variables:

1. t: discrete time step, already defined in the problem as 1,2,...,10.

2. A(t): inflow of water at point A at time t, defined by the function A(t) = f(t) = max(-4/30t^3+t^2-0.234t+3, 0).

3. B(t), C(t), D(t), E(t), F(t): flow of water in points B, C, D, E, F at time t, derived from the flow and pipes configuration.

4. AG(t), CG(t), FG(t): outflow of water from points A, C, F at time t, with possible values of 0 or more, based on the excess water that can't be transported via the pipes.

5. Pipe_XY(t): flow of water in pipe from point X to point Y at time t, depending on the activation status of the pipe and its capacity.

6. Activation_XY: activation status of pipe from point X to point Y, either 0 (inactive) or 1(active).

7. Pipe_capacity_XY: capacity of pipe from point X to point Y, defined in the problem as fixed values for each pipe.

8. Num_pipes_active: Count of the number of active pipes, subjected to the constraint of being less than or equal to 10.


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [8]:
print(response2.choices[0].message.content)

Objective Function:

Maximize G(t) = ∑ from t=1 to t=10 (Pipe_EG(t))

Where:

- G(t) is the water flow at point G at time step t.

Subject to the constraints involving pipe capacities and activation status. The specific form of these constraints is left to the assistant.


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [11]:
print(response3.choices[0].message.content)

Constraints:

1. Pipe_XY(t) <= Pipe_capacity_XY, for all t and for all pipes XY
2. A(t) = B(𝑡−1) + C(𝑡−1) + F(𝑡−1), for all t from 2 to 10
3. B(t) = Pipe_AB(t) + Pipe_FB(t), for all t
4. C(t) = Pipe_AC(t) + Pipe_BC(t), for all t
5. D(t) = Pipe_BD(t), for all t
6. E(t) = Pipe_BE(t) + Pipe_CE(t) + Pipe_DE(t), for all t
7. F(t) = Pipe_AF(t) + Pipe_CF(t), for all t
8. G(t) = G(t−1) + Pipe_EG(t), for all t from 2 to 10
9. ∑ Activation_XY over all pipes XY <= 10
10. If Activation_XY = 0, then Pipe_XY(t) = 0, for all t and for all pipes XY
11. A(t+1) = max(-4/30*(t+1)^3 + (t+1)^2 - 0.234*(t+1) + 3, 0), for all t from 1 to 9
12. AG(t), CG(t), FG(t) >= 0, for all t
13. G(1) = Pipe_EG(1)

Note: Pipes are assumed to be have been labeled according to this configuration: AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG.


### **Generate Pyomo Code**

In [12]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [13]:
Markdown(response4.choices[0].message.content)

```python
from pyomo.environ import *

# Sample Data
pipe_capacity = {'AB': 5, 'AC': 4, 'AF': 6, 'BC': 2, 'BD': 3, 
                 'BE': 1, 'CE': 3, 'CF': 4, 'DE': 2, 'EG': 3, 'FB': 2, 'FG': 2}
time_periods = list(range(1,11))
pipes = list(pipe_capacity.keys())

# Create Model
model = ConcreteModel()

# Variables
model.flow = Var(time_periods, pipes, domain=NonNegativeReals)
model.activation = Var(pipes, domain=Binary)

# Objective
model.obj = Objective(expr=sum(model.flow[i, 'EG'] for i in time_periods), sense=maximize)

# Constraints
model.constraints = ConstraintList()
for t in time_periods:
    for p in pipes:
        model.constraints.add(model.flow[t, p] <= model.activation[p]*pipe_capacity[p])

for t in time_periods[1:]:
    model.constraints.add(sum( model.flow[t-1, out] for out in ['AB', 'AC', 'AF'] if (t-1, out) in model.flow) == \
                          sum( model.flow[t, inc] for inc in ['BC', 'BE', 'BD', 'EG', 'FG'] if (t, inc) in model.flow))

for t in time_periods:
    model.constraints.add(sum(model.flow[t, p] for p in pipes if (t,p) in model.flow) <= max(-4/30*t**3+t**2-0.234*t+3, 0))

model.constraints.add(sum(model.activation[p] for p in pipes) <= 10)

for t in time_periods:
    for p in pipes:
        model.constraints.add(model.flow[t, p] <= pipe_capacity[p])

# solver
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for v in model.component_objects(Var, active=True):
    print("Variable",v)
    varobject = getattr(model, str(v))
    for index in varobject:
        print(" ",index, varobject[index].value)
```

### **Run the code Generated by GPT4**

In [14]:
from pyomo.environ import *

# Sample Data
pipe_capacity = {'AB': 5, 'AC': 4, 'AF': 6, 'BC': 2, 'BD': 3,
                 'BE': 1, 'CE': 3, 'CF': 4, 'DE': 2, 'EG': 3, 'FB': 2, 'FG': 2}
time_periods = list(range(1,11))
pipes = list(pipe_capacity.keys())

# Create Model
model = ConcreteModel()

# Variables
model.flow = Var(time_periods, pipes, domain=NonNegativeReals)
model.activation = Var(pipes, domain=Binary)

# Objective
model.obj = Objective(expr=sum(model.flow[i, 'EG'] for i in time_periods), sense=maximize)

# Constraints
model.constraints = ConstraintList()
for t in time_periods:
    for p in pipes:
        model.constraints.add(model.flow[t, p] <= model.activation[p]*pipe_capacity[p])

for t in time_periods[1:]:
    model.constraints.add(sum( model.flow[t-1, out] for out in ['AB', 'AC', 'AF'] if (t-1, out) in model.flow) == \
                          sum( model.flow[t, inc] for inc in ['BC', 'BE', 'BD', 'EG', 'FG'] if (t, inc) in model.flow))

for t in time_periods:
    model.constraints.add(sum(model.flow[t, p] for p in pipes if (t,p) in model.flow) <= max(-4/30*t**3+t**2-0.234*t+3, 0))

model.constraints.add(sum(model.activation[p] for p in pipes) <= 10)

for t in time_periods:
    for p in pipes:
        model.constraints.add(model.flow[t, p] <= pipe_capacity[p])

# solver
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for v in model.component_objects(Var, active=True):
    print("Variable",v)
    varobject = getattr(model, str(v))
    for index in varobject:
        print(" ",index, varobject[index].value)

Variable flow
  (1, 'AB') 0.632666666666667
  (1, 'AC') 0.0
  (1, 'AF') 0.0
  (1, 'BC') 0.0
  (1, 'BD') 0.0
  (1, 'BE') 0.0
  (1, 'CE') 0.0
  (1, 'CF') 0.0
  (1, 'DE') 0.0
  (1, 'EG') 3.0
  (1, 'FB') 0.0
  (1, 'FG') 0.0
  (2, 'AB') 3.0
  (2, 'AC') 0.0
  (2, 'AF') 0.0
  (2, 'BC') 0.0
  (2, 'BD') 0.0
  (2, 'BE') 0.0
  (2, 'CE') 0.0
  (2, 'CF') 0.0
  (2, 'DE') 0.0
  (2, 'EG') 0.632666666666667
  (2, 'FB') 0.0
  (2, 'FG') 0.0
  (3, 'AB') 3.0
  (3, 'AC') 0.0
  (3, 'AF') 0.0
  (3, 'BC') 0.0
  (3, 'BD') 0.0
  (3, 'BE') 0.0
  (3, 'CE') 0.0
  (3, 'CF') 0.0
  (3, 'DE') 0.0
  (3, 'EG') 3.0
  (3, 'FB') 0.0
  (3, 'FG') 0.0
  (4, 'AB') 3.0
  (4, 'AC') 0.0
  (4, 'AF') 0.0
  (4, 'BC') 0.0
  (4, 'BD') 0.0
  (4, 'BE') 0.0
  (4, 'CE') 0.0
  (4, 'CF') 0.0
  (4, 'DE') 0.0
  (4, 'EG') 3.0
  (4, 'FB') 0.0
  (4, 'FG') 0.0
  (5, 'AB') 3.0
  (5, 'AC') 0.0
  (5, 'AF') 0.0
  (5, 'BC') 0.0
  (5, 'BD') 0.0
  (5, 'BE') 0.0
  (5, 'CE') 0.0
  (5, 'CF') 0.0
  (5, 'DE') 0.0
  (5, 'EG') 3.0
  (5, 'FB') 0.0
  (5, 'FG') 0.

In [17]:
from pyomo.environ import *

# Sample Data
pipe_capacity = {'AB': 3, 'AC': 6, 'AF': 1, 'BC': 3, 'BD': 10,
                 'BE': 4, 'CE': 4, 'CF': 4, 'DE': 5, 'EG': 5, 'FB': 12 ,'FG': 7}
time_periods = list(range(1,11))
pipes = list(pipe_capacity.keys())

# Create Model
model = ConcreteModel()

# Variables
model.flow = Var(time_periods, pipes, domain=NonNegativeReals)
model.activation = Var(pipes, domain=Binary)

# Objective
model.obj = Objective(expr=sum(model.flow[i, 'EG'] for i in time_periods), sense=maximize)

# Constraints
model.constraints = ConstraintList()
for t in time_periods:
    for p in pipes:
        model.constraints.add(model.flow[t, p] <= model.activation[p]*pipe_capacity[p])

for t in time_periods[1:]:
    model.constraints.add(sum( model.flow[t-1, out] for out in ['AB', 'AC', 'AF'] if (t-1, out) in model.flow) == \
                          sum( model.flow[t, inc] for inc in ['BC', 'BE', 'BD', 'EG', 'FG'] if (t, inc) in model.flow))

for t in time_periods:
    model.constraints.add(sum(model.flow[t, p] for p in pipes if (t,p) in model.flow) <= max(-4/30*t**3+t**2-0.234*t+3, 0))

model.constraints.add(sum(model.activation[p] for p in pipes) <= 10)

for t in time_periods:
    for p in pipes:
        model.constraints.add(model.flow[t, p] <= pipe_capacity[p])

# solver
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for v in model.component_objects(Var, active=True):
    print("Variable",v)
    varobject = getattr(model, str(v))
    for index in varobject:
        print(" ",index, varobject[index].value)
print(model.obj())

Variable flow
  (1, 'AB') 0.0
  (1, 'AC') 0.465333333333334
  (1, 'AF') 0.0
  (1, 'BC') 0.0
  (1, 'BD') 0.0
  (1, 'BE') 0.0
  (1, 'CE') 0.0
  (1, 'CF') 0.0
  (1, 'DE') 0.0
  (1, 'EG') 3.16733333333333
  (1, 'FB') 0.0
  (1, 'FG') 0.0
  (2, 'AB') 0.0
  (2, 'AC') 5.0
  (2, 'AF') 0.0
  (2, 'BC') 0.0
  (2, 'BD') 0.0
  (2, 'BE') 0.0
  (2, 'CE') 0.0
  (2, 'CF') 0.0
  (2, 'DE') 0.0
  (2, 'EG') 0.465333333333334
  (2, 'FB') 0.0
  (2, 'FG') 0.0
  (3, 'AB') 0.0
  (3, 'AC') 2.698
  (3, 'AF') 0.0
  (3, 'BC') 0.0
  (3, 'BD') 0.0
  (3, 'BE') 0.0
  (3, 'CE') 0.0
  (3, 'CF') 0.0
  (3, 'DE') 0.0
  (3, 'EG') 5.0
  (3, 'FB') 0.0
  (3, 'FG') 0.0
  (4, 'AB') 0.0
  (4, 'AC') 5.0
  (4, 'AF') 0.0
  (4, 'BC') 0.0
  (4, 'BD') 0.0
  (4, 'BE') 0.0
  (4, 'CE') 0.0
  (4, 'CF') 0.0
  (4, 'DE') 0.0
  (4, 'EG') 2.698
  (4, 'FB') 0.0
  (4, 'FG') 0.0
  (5, 'AB') 0.0
  (5, 'AC') 4.16733333333333
  (5, 'AF') 0.0
  (5, 'BC') 0.0
  (5, 'BD') 0.0
  (5, 'BE') 0.0
  (5, 'CE') 0.0
  (5, 'CF') 0.0
  (5, 'DE') 0.0
  (5, 'EG') 5.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**